# Experimentos Juego del Gato y el Ratón

Este notebook ejecuta y analiza **experimentos automáticos** del juego de Gato y Ratón, integrando en un único flujo:

- La batería de *pairings* específica (como en `experiments_runner.py`).
- La exploración sistemática de **todas las combinaciones de algoritmos** en **dos mapas** (small y big), como en el notebook de registro de resultados.

A lo largo del notebook se generan y visualizan todos los gráficos necesarios para el informe, listos para ser exportados o reutilizados.


## 1. Imports y configuración general

In [ ]:

import random
import time
import statistics as st
from collections import Counter
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

# Mapas
from scr.config_small import conexiones as conexiones_small, nodos as nodos_small
from scr.config_big import conexiones as conexiones_big, nodos as nodos_big

# Algoritmos
from scr.alg.alg_astar import gato_move_astar, raton_move_astar
from scr.alg.alg_minimax import gato_move_minimax, raton_move_minimax
from scr.alg.alg_random import gato_move_random, raton_move_random
from scr.alg.alg_entrenamiento import bfs_dist

print("Módulos importados correctamente.")


## 2. Parámetros globales de los experimentos

En esta sección se definen los parámetros compartidos por ambas baterías de experimentos: número máximo de pasos por partida, combinaciones de algoritmos, etc.

In [ ]:

# Número máximo de pasos por partida
MAX_PASOS = 200

# Número de episodios por combinación de algoritmos y mapa
EPISODIOS_POR_SETUP = 50

# Algoritmos disponibles para cada agente (para la batería 'todos contra todos') 
ALGORITMOS_GATO = ["random", "astar", "minimax"]
ALGORITMOS_RATON = ["random", "astar", "minimax"]

# Pairings específicos (batería estilo experiments_runner.py, usando mapa small por defecto)
PAIRINGS = [
    ("G_random vs R_astar", "random", "astar"),
    ("G_astar vs R_random", "astar", "random"),
    ("G_minmax vs R_astar", "minimax", "astar"),
    ("G_random vs R_minmax", "random", "minimax"),
    ("G_astar vs R_minmax", "astar", "minimax"),
    ("G_minmax vs R_random", "minimax", "random"),
]

MAX_PASOS, EPISODIOS_POR_SETUP, ALGORITMOS_GATO, ALGORITMOS_RATON, PAIRINGS


## 3. Funciones de movimiento y utilidades

Definimos funciones auxiliares para mover al gato y al ratón según el algoritmo elegido, así como la inicialización de una partida (posiciones iniciales, queso y meta).

In [ ]:

def mover_gato(mode, pos_g, pos_r, conexiones, nodos):
    """Devuelve la nueva posición del gato según el modo seleccionado."""
    if mode == "astar":
        return gato_move_astar(conexiones, nodos, pos_g, pos_r)
    elif mode == "minimax":
        return gato_move_minimax(conexiones, nodos, pos_g, pos_r)
    elif mode == "random":
        return gato_move_random(conexiones, pos_g, pos_r)
    else:
        raise ValueError(f"Modo gato desconocido: {mode}")


def mover_raton(mode, pos_g, pos_r, queso, meta, tiene_queso, conexiones, nodos):
    """Devuelve la nueva posición del ratón según el modo seleccionado."""
    if mode == "astar":
        return raton_move_astar(conexiones, nodos, pos_g, pos_r, queso, meta, tiene_queso)
    elif mode == "minimax":
        # En este proyecto, minimax ignora queso/meta
        return raton_move_minimax(conexiones, nodos, pos_g, pos_r)
    elif mode == "random":
        return raton_move_random(conexiones, pos_g, pos_r)
    else:
        raise ValueError(f"Modo ratón desconocido: {mode}")


def inicializar_partida(conexiones, nodos, seed=None):
    """Elige al azar posiciones iniciales válidas para ratón, gato, queso y meta."""
    if seed is not None:
        random.seed(seed)
    nodos_ids = list(nodos.keys())
    pos_raton = random.choice(nodos_ids)
    pos_gato  = random.choice([n for n in nodos_ids if n != pos_raton])
    queso     = random.choice([n for n in nodos_ids if n not in (pos_raton, pos_gato)])
    meta      = random.choice([n for n in nodos_ids if n not in (pos_raton, pos_gato, queso)])

    return pos_gato, pos_raton, queso, meta


## 4. Simulación básica de una partida (winner, pasos, duración)

Esta función replica la lógica de `experiments_runner.py`, devolviendo solo el ganador, la cantidad de pasos y la duración en segundos. Se utilizará para la batería de *pairings*.

In [ ]:

def simular_un_juego(mode_gato, mode_raton, conexiones, nodos, max_pasos=200, seed=None):
    """
    Simula una partida completa y devuelve un dict con claves:
      - 'winner': 'gato' | 'raton' | 'empate'
      - 'pasos': número de turnos utilizados
      - 'duration': tiempo de simulación (wall-clock) en segundos
    """
    pos_gato, pos_raton, queso, meta = inicializar_partida(conexiones, nodos, seed=seed)
    tiene_queso = False
    turno = "raton"
    pasos = 0

    t0 = time.perf_counter()

    while pasos < max_pasos:
        if turno == "raton":
            pos_raton = mover_raton(mode_raton, pos_gato, pos_raton, queso, meta, tiene_queso, conexiones, nodos)
            # recoge queso
            if (not tiene_queso) and (pos_raton == queso):
                tiene_queso = True
            turno = "gato"
        else:
            pos_gato = mover_gato(mode_gato, pos_gato, pos_raton, conexiones, nodos)
            turno = "raton"

        pasos += 1

        # condiciones terminales
        if pos_gato == pos_raton:
            duration = time.perf_counter() - t0
            return {"winner": "gato", "pasos": pasos, "duration": duration}

        if tiene_queso and pos_raton == meta:
            duration = time.perf_counter() - t0
            return {"winner": "raton", "pasos": pasos, "duration": duration}

    duration = time.perf_counter() - t0
    return {"winner": "empate", "pasos": pasos, "duration": duration}


# Prueba rápida
simular_un_juego("astar", "random", conexiones_small, nodos_small, max_pasos=50, seed=123)


## 5. Simulación detallada de una partida (captura, escape, empate, distancia promedio)

Para el análisis más fino se utiliza una función que, además de la dinámica básica, registra si la partida terminó por **captura del ratón**, **escape** (ratón llega a la meta con queso) o **empate** (se agotan los pasos), y calcula la **distancia promedio gato–ratón** durante la partida usando `bfs_dist`.

In [ ]:

def simular_partida(modo_g, modo_r, conexiones, nodos, max_pasos=200, seed=None):
    """
    Simula una partida y devuelve:
      - modo_gato, modo_raton
      - captura (1 si gana el gato)
      - escape  (1 si gana el ratón escapando con queso)
      - empate  (1 si nadie gana dentro de MAX_PASOS)
      - pasos   (pasos utilizados)
      - d_prom  (distancia promedio gato–ratón a lo largo de la partida)
    """
    pos_gato, pos_raton, queso, meta = inicializar_partida(conexiones, nodos, seed=seed)
    tiene_queso = False
    turno = "raton"
    pasos = 0

    captura = 0
    escape = 0
    empate = 0
    d_hist = []

    while pasos < max_pasos:
        # registrar distancia actual
        try:
            d = bfs_dist(conexiones, pos_gato, pos_raton)
        except Exception:
            # si hubiera algún problema con bfs_dist, registramos 0 como fallback
            d = 0
        d_hist.append(d)

        if turno == "raton":
            pos_raton = mover_raton(modo_r, pos_gato, pos_raton, queso, meta, tiene_queso, conexiones, nodos)
            if (not tiene_queso) and (pos_raton == queso):
                tiene_queso = True
            turno = "gato"
        else:
            pos_gato = mover_gato(modo_g, pos_gato, pos_raton, conexiones, nodos)
            turno = "raton"

        pasos += 1

        # condiciones terminales
        if pos_gato == pos_raton:
            captura = 1
            break

        if tiene_queso and pos_raton == meta:
            escape = 1
            break

    if not (captura or escape):
        empate = 1

    return {
        "modo_gato": modo_g,
        "modo_raton": modo_r,
        "captura": captura,
        "escape": escape,
        "empate": empate,
        "pasos": pasos,
        "d_prom": st.mean(d_hist) if d_hist else 0.0,
    }


# Prueba rápida
simular_partida("astar", "random", conexiones_small, nodos_small, max_pasos=50, seed=123)


## 6. Batería de *pairings*

En esta sección se ejecutan los **seis emparejamientos fijos** de algoritmos para gato y ratón sobre el mapa **small**, repitiendo cada configuración varias veces con distintas semillas.

Se registran:

- Ganador de la partida (`gato`, `raton`, `empate`).
- Número de pasos hasta el final.
- Tiempo de simulación en segundos.


In [ ]:

def correr_pairings(pairings, conexiones, nodos, repetitions=20, seed_base=0, max_pasos=200):
    resultados = []
    for idx, (label, mg, mr) in enumerate(pairings):
        print(f"Pairing {idx+1}/{len(pairings)}: {label}")
        for i in range(repetitions):
            seed = seed_base + 1000*idx + i
            out = simular_un_juego(mg, mr, conexiones, nodos, max_pasos=max_pasos, seed=seed)
            out["pair"] = label
            out["mode_gato"] = mg
            out["mode_raton"] = mr
            out["seed"] = seed
            resultados.append(out)
    return pd.DataFrame(resultados)


df_pairings = correr_pairings(PAIRINGS, conexiones_small, nodos_small,
                                       repetitions=20, seed_base=0, max_pasos=MAX_PASOS)

out_dir = Path("resultados")
out_dir.mkdir(exist_ok=True)
df_pairings.to_csv(out_dir / "resultados_partidas_detalle.csv", index=False)
df_pairings.head()


### 6.1 Resumen y gráficos de los *pairings*

Se calcula la tasa de victoria del gato, del ratón y de empates para cada pairing, junto con la duración promedio de las partidas.

Los siguientes gráficos replican la idea de `win_rates.png` y `avg_duration.png`, pero mostrados directamente en el notebook.

In [ ]:

# Resumen por pairing
resumen_rows = []
for label, group in df_pairings.groupby("pair"):
    winners = Counter(group["winner"])
    total = len(group)
    tasa_gato = winners.get("gato", 0) / total
    tasa_raton = winners.get("raton", 0) / total
    tasa_empate = winners.get("empate", 0) / total
    pasos_prom = group["pasos"].mean()
    dur_prom = group["duration"].mean()

    resumen_rows.append({
        "pair": label,
        "tasa_gato": tasa_gato,
        "tasa_raton": tasa_raton,
        "tasa_empate": tasa_empate,
        "pasos_prom": pasos_prom,
        "dur_prom": dur_prom,
    })

df_pairings_resumen = pd.DataFrame(resumen_rows)
df_pairings_resumen.to_csv(out_dir / "resultados_partidas_pairing.csv", index=False)
df_pairings_resumen


In [ ]:

# 1) Win rates (gato / ratón / empate) por pairing
labels = df_pairings_resumen["pair"].tolist()
gato_rates = df_pairings_resumen["tasa_gato"].tolist()
raton_rates = df_pairings_resumen["tasa_raton"].tolist()
empate_rates = df_pairings_resumen["tasa_empate"].tolist()

x = range(len(labels))
plt.figure(figsize=(10, 6))
plt.bar(x, gato_rates, label="Gato")
plt.bar(x, raton_rates, bottom=gato_rates, label="Ratón")
bottom2 = [g + r for g, r in zip(gato_rates, raton_rates)]
plt.bar(x, empate_rates, bottom=bottom2, label="Empate")
plt.xticks(list(x), labels, rotation=30, ha="right")
plt.ylabel("Tasa")
plt.title("Win rates por pairing (mapa small)")
plt.legend()
plt.tight_layout()
plt.show()

# 2) Duración promedio por pairing (segundos)
plt.figure(figsize=(10, 5))
plt.bar(labels, df_pairings_resumen["dur_prom"].tolist())
plt.xticks(rotation=30, ha="right")
plt.ylabel("Duración promedio (s)")
plt.title("Duración promedio de las partidas por pairing (mapa small)")
plt.tight_layout()
plt.show()


## 7. Batería completa en ambos mapas (todos los algoritmos)

Ahora se replica la lógica del notebook de **registro de resultados**, ejecutando **todas las combinaciones de algoritmos** para gato y ratón (`random`, `astar`, `minimax`) en los mapas `small` y `big`.

Para cada partida se registra si terminó por captura, escape o empate, junto con los pasos y la distancia promedio gato–ratón.

In [ ]:

resultados_detalle = []

def correr_experimentos_para_mapa(nombre_mapa, conexiones, nodos):
    """Ejecuta todas las combinaciones de algoritmos en el mapa dado."""
    print(f"\n===== MAPA: {nombre_mapa} =====")
    for mg in ALGORITMOS_GATO:
        for mr in ALGORITMOS_RATON:
            print(f"Gato={mg} vs Ratón={mr}")
            for ep in range(EPISODIOS_POR_SETUP):
                seed = 10_000 + ep
                res = simular_partida(mg, mr, conexiones, nodos,
                                      max_pasos=MAX_PASOS,
                                      seed=seed)
                res["mapa"] = nombre_mapa
                resultados_detalle.append(res)

# Ejecutar en ambos mapas
correr_experimentos_para_mapa("small", conexiones_small, nodos_small)
correr_experimentos_para_mapa("big", conexiones_big, nodos_big)

df = pd.DataFrame(resultados_detalle)
df.head()


### 7.1 Resumen estadístico por mapa y combinación de algoritmos

Se agrupan los resultados por mapa y por configuración (modo del gato y modo del ratón), obteniendo las tasas promedio de captura, escape y empate, así como los pasos medios y la distancia promedio gato–ratón.

In [ ]:

# Resumen por mapa + combinación de algoritmos
df_group = (
    df.groupby(["mapa", "modo_gato", "modo_raton"]).agg({
        "captura": "mean",
        "escape": "mean",
        "empate": "mean",
        "pasos": "mean",
        "d_prom": "mean",
    }).reset_index()
)

df_group


### 7.2 Exportación de resultados a CSV

Se exportan dos archivos:

- `resultados_partidas_detalle.csv`: una fila por partida.
- `resultados_partidas_resumen.csv`: una fila por combinación (mapa, modo gato, modo ratón).

In [ ]:

df.to_csv("resultados_partidas_detalle.csv", index=False)
df_group.to_csv("resultados_partidas_pairing.csv", index=False)
"Archivos CSV guardados en el directorio actual."


### 7.3 Visualización de resultados por mapa

Los siguientes gráficos replican la estructura del notebook de registro de resultados:

1. **Tasas de resultado** (captura, escape, empate) por configuración de algoritmos.
2. **Tasa de empate y pasos promedio** por configuración.

Se generan gráficos separados para cada mapa (`small` y `big`).

In [ ]:

# Función auxiliar para armar etiqueta compacta
def etiqueta_fila(row):
    return f"G:{row['modo_gato']}-R:{row['modo_raton']}"

for mapa in df_group["mapa"].unique():
    sub = df_group[df_group["mapa"] == mapa].copy()
    etiquetas = sub.apply(etiqueta_fila, axis=1)
    x = range(len(sub))

    # Gráfico de barras apiladas: captura / escape / empate
    plt.figure(figsize=(10, 6))
    plt.title(f"Tasas de resultado por configuración - mapa {mapa}")
    plt.xticks(list(x), etiquetas, rotation=45, ha="right")
    plt.ylabel("Proporción sobre partidas")

    plt.bar(x, sub["captura"], label="Captura gato")
    plt.bar(x, sub["escape"], bottom=sub["captura"], label="Escape ratón")
    bottom_emp = sub["captura"] + sub["escape"]
    plt.bar(x, sub["empate"], bottom=bottom_emp, label="Empate")

    plt.legend()
    plt.tight_layout()
    plt.show()

    # Gráfico de líneas: empate y pasos promedio
    plt.figure(figsize=(10, 5))
    plt.title(f"Empate y pasos promedio — mapa {mapa}")
    plt.xticks(list(x), etiquetas, rotation=45, ha="right")
    plt.ylabel("Empate / Pasos")

    plt.plot(x, sub["empate"], marker="o", label="Tasa de empate")
    plt.plot(x, sub["pasos"], marker="s", label="Pasos promedio")

    plt.legend()
    plt.tight_layout()
    plt.show()
